In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('Data/archive/rating.csv')

In [17]:
df['userId'] = df['userId'].astype('int32')
df['movieId'] = df['movieId'].astype('int32')
df['rating'] = df['rating'].astype('float32')

df.drop(['timestamp'], axis=1, inplace=True)
df.head(2)

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5


In [18]:
df_movies = pd.read_csv('Data/archive/movie.csv')

In [5]:
from tqdm import tqdm
# Group by userId and collect indices for the test set
test_indices = []
extra_indices = []
for user, group in tqdm(df.groupby('userId')):
    if len(group) > 1:
        # Sample 20% of the user's data, with a minimum of 1 entry
        sample_indices = group.sample(max(1, round(len(group) * 0.2)), random_state=42).index
    else:
        print("batatata")
        # Take all the data if the user has less than 2 entries
        # sample_indices = group.index
        # extra_indices.extend(sample_indices)
    
    test_indices.extend(sample_indices)

# Convert test_indices to a numpy array
test_indices = np.array(test_indices)
#extra_indices = np.array(extra_indices)


100%|██████████| 138493/138493 [00:50<00:00, 2715.67it/s]


In [6]:
test_dataset = df.loc[test_indices]  
# Create the train dataset by excluding the test indices
train_dataset = df.drop(test_indices)


In [7]:
pd.merge(test_dataset,train_dataset,how='inner',on=['userId','movieId'])

,userId,movieId,rating_x,rating_y


In [8]:
test_dataset.reset_index(drop=True, inplace=True)

In [9]:
train_dataset.reset_index(drop=True, inplace=True)

In [19]:
def create_interaction_matrix (df,movies_list):
    # Extract unique userIds and movieIds
    unique_users = df['userId'].unique()
    unique_movies = movies_list
    # Create a mapping from userId and movieId to indices
    user_map = {user: idx for idx, user in enumerate(unique_users)}
    movie_map = {movie: idx for idx, movie in enumerate(unique_movies)}

    # Initialize the matrix with NaN (or any default value, e.g., 0)
    pivoted_matrix = np.full((len(unique_users), len(unique_movies)), 0,dtype=np.int8)

    # Populate the matrix with the ratings
    for row in df.itertuples():
        user_idx = user_map[row.userId]
        movie_idx = movie_map[row.movieId]
        pivoted_matrix[user_idx, movie_idx] = 1 if row.rating >= 4 else 0
    # Display the first 2 rows
    print(pivoted_matrix[:2, :])
    return pivoted_matrix


In [20]:
interaction_matrix = create_interaction_matrix(df,df['movieId'].unique())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [21]:
interaction_matrix.shape

(138493, 26744)

In [22]:
np.save('Data/interaction_matrix.npy', interaction_matrix)

In [11]:
train_interaction_matrix= create_interaction_matrix(train_dataset,df['movieId'].unique())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
test_interaction_matrix= create_interaction_matrix(test_dataset,df['movieId'].unique())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
np.save('Data/train_rating_matrix.npy',train_interaction_matrix)
np.save('Data/test_rating_matrix.npy',test_interaction_matrix)

In [14]:
test_interaction_matrix.shape,train_interaction_matrix.shape

((138493, 26744), (138493, 26744))

## item features